<a href="https://colab.research.google.com/github/JoaoDaross/CursoAwari/blob/main/ColetaDadosMercadoLivre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coletando dados de zonas e páginas do mercado livre e salvando em um dataframe

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from time import sleep

In [2]:
zonas = ['sul', 'norte', 'leste', 'oeste']
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241']

In [3]:
url_ml = 'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}'

In [4]:
def funcao_zona(zona):
    lista_zonas = []
    lista_zonas.append(zona)

In [5]:
for zona in zonas:
    for pagina in paginas:
        funcao_zona(zona)
        print(pagina, zona)

 sul
_Desde_49 sul
_Desde_97 sul
_Desde_145 sul
_Desde_193 sul
_Desde_241 sul
 norte
_Desde_49 norte
_Desde_97 norte
_Desde_145 norte
_Desde_193 norte
_Desde_241 norte
 leste
_Desde_49 leste
_Desde_97 leste
_Desde_145 leste
_Desde_193 leste
_Desde_241 leste
 oeste
_Desde_49 oeste
_Desde_97 oeste
_Desde_145 oeste
_Desde_193 oeste
_Desde_241 oeste


In [6]:
re_quarto ='\| (.*) quarto'
re_area = '> (.*) m²'

In [19]:
def coleta_dados(url, zona):
  conteudo = requests.get(url).content
  sopa = BeautifulSoup(conteudo)

  lista_precos = []
  lista_areas_quartos = []
  lista_quartos = []
  lista_dimensoes = []
  lista_zonas = []

  re_quartos = '(.) quarto|quartos'
  re_dimensoes = '(.*) m²'

  precos = sopa.find_all('span', class_='price-tag-fraction')
  dimensoes_e_quartos = sopa.find_all('ul', class_='ui-search-card-attributes ui-search-item__group__element')

  for elemento in precos:
    preco = re.findall('<span class="price-tag-fraction">(.*)</span>', str(elemento))
    lista_precos.append(float(preco[0])*1000)   

  for elemento in dimensoes_e_quartos:
    lista_areas_quartos.append(elemento.text)

  for elemento in lista_areas_quartos:
    quarto = re.findall(re_quartos, str(elemento))
    dimensao = re.findall(re_dimensoes, str(elemento).replace(',', ''))

    if quarto == []:
      quarto = np.nan
    else:
      quarto = quarto[0]
    
    if dimensao == []:
      dimensao = np.nan
    else:
      dimensao = dimensao[0]
  
    lista_quartos.append(quarto)
    lista_dimensoes.append(dimensao)
    lista_zonas.append(zona)

  dados = {'zona': lista_zonas,
           'precos':lista_precos,
           'quartos':lista_quartos,
           'areas': lista_dimensoes}
  
  df = pd.DataFrame(dados)
  return df

In [20]:
lista_dfs = []

for zona in zonas:
  for pagina in paginas:
    url = f'https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-{zona}/{pagina}'
    print('Coletando dados de', url)
    df = coleta_dados(url, zona)
    lista_dfs.append(df)
    sleep(2)
print('Dados coletados!')

Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_193
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_241
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_97
Coletando dados de https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-nor

In [21]:
df = pd.concat(lista_dfs)

In [23]:
df.head()

,zona,precos,quartos,areas
0,sul,500000.0,2,51
1,sul,1555.0,2,80
2,sul,800000.0,2,51
3,sul,20000.0,NaN,386
4,sul,2500.0,3,105


In [24]:
df.tail()

,zona,precos,quartos,areas
43,oeste,2300.0,1,32
44,oeste,610000.0,NaN,42
45,oeste,1050.0,1,30
46,oeste,800000.0,NaN,30
47,oeste,1500.0,2,49


In [25]:
df.shape

(1152, 4)

In [27]:
df.to_csv('dados_mercado_livre.csv', index=False)

In [28]:
df = pd.read_csv('dados_mercado_livre.csv')

In [29]:
type(dados)

pandas.core.frame.DataFrame

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   zona     1152 non-null   object 
 1   precos   1152 non-null   float64
 2   quartos  1122 non-null   float64
 3   areas    1149 non-null   float64
dtypes: float64(3), object(1)
memory usage: 36.1+ KB


In [31]:
df.count()

zona       1152
precos     1152
quartos    1122
areas      1149
dtype: int64

In [32]:
df.isnull().sum()

zona        0
precos      0
quartos    30
areas       3
dtype: int64

In [33]:
df.shape

(1152, 4)